# Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import lightgbm as lgb
import tqdm
import gc
%matplotlib inline

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
ab = pd.read_csv('dota2_abilities.csv', index_col='ability_id')
items = pd.read_csv('dota2_items.csv', index_col='item_id')
her = pd.read_csv('dota2_heroes.csv', index_col='hero_id')
train = pd.read_csv('academy2019_final_train.csv', index_col='id')
test = pd.read_csv('academy2019_final_test.csv', index_col='id')
y = train["skilled"]

train = train.drop('skilled', axis=1)

In [3]:
index_train = train.index 
index_test = test.index 

In [4]:
ab.behavior = ab.behavior.fillna('empty')
ab['behavior_first'] = ab.behavior.apply(lambda x: x.split("'")).apply(lambda x: x[1] if len(x) > 1 else 'empty')
her.roles = her.roles.apply(lambda x: x.split("'")).apply(lambda x: x[1] if len(x) > 1 else 'unknown')
items.qual = items.qual.fillna('empty')

# JSON, slowly

In [5]:
component_t=[]
secret_shop_t=[]
consumable_t = []
common_t= [] 
rare_t= []
epic_t = []
artifact_t = [] 
empty_t = []

with open('academy2019_final_test.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        table = {'component': 0, 'secret_shop': 0,  'consumable': 0, 'common': 0, 'rare': 0,
       'epic': 0, 'artifact': 0, 'empty': 0}
        record = json.loads(line)
        for item in record['item_purchase_log']:
            item = item['item_id']
            table[items.loc[item, 'qual']] += 1
        component_t.append(table['component'])
        secret_shop_t.append(table['secret_shop'])
        consumable_t.append(table['consumable'])
        rare_t.append(table['rare'])
        epic_t.append(table['epic'])
        artifact_t.append(table['artifact'])
        common_t.append(table['common'])
        empty_t.append(table['empty'])
        
        
        
        
        
component=[]
secret_shop=[]
consumable = []
common= [] 
rare= []
epic = []
artifact = [] 
empty = []

with open('academy2019_final_train.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        table = {'component': 0, 'secret_shop': 0,  'consumable': 0, 'common': 0, 'rare': 0,
       'epic': 0, 'artifact': 0, 'empty': 0}
        record = json.loads(line)
        for item in record['item_purchase_log']:
            item = item['item_id']
            table[items.loc[item, 'qual']] += 1
        component.append(table['component'])
        secret_shop.append(table['secret_shop'])
        consumable.append(table['consumable'])
        rare.append(table['rare'])
        epic.append(table['epic'])
        artifact.append(table['artifact'])
        common.append(table['common'])
        empty.append(table['empty'])

A Jupyter Widget

A Jupyter Widget

In [8]:
for localized_name in set(her.localized_name):
    train['against_{}'.format(localized_name)] = 0
    test['against_{}'.format(localized_name)] = 0
    train['with_{}'.format(localized_name)] = 0
    test['with_{}'.format(localized_name)] = 0
    

with open('academy2019_final_train.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        names = ['radiant', 'dire']
        record = json.loads(line)
        name = record['player_team']
        names.remove(name)
        for hero in record[names[0]+'_heroes']:
             train.loc[record['id'], 'against_{}'.format(her.loc[hero, 'localized_name'])] += 1
        for hero in record[name +'_heroes']:
            train.loc[record['id'], 'with_{}'.format(her.loc[hero, 'localized_name'])] += 1
 

with open('academy2019_final_test.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        names = ['radiant', 'dire']
        record = json.loads(line)
        name = record['player_team']
        names.remove(name)
        for hero in record[names[0]+'_heroes']:
             test.loc[record['id'], 'against_{}'.format(her.loc[hero, 'localized_name'])] += 1
        for hero in record[name +'_heroes']:
            test.loc[record['id'], 'with_{}'.format(her.loc[hero, 'localized_name'])] += 1

A Jupyter Widget

A Jupyter Widget

In [9]:
for ability_behavior in set(ab.behavior_first.fillna('empty')):
    train['upgrade_behavior {}'.format(ability_behavior)] = 0
    test['upgrade_behavior {}'.format(ability_behavior)] = 0
    
with open('academy2019_final_train.jsonlines') as fin:
    n_upgrades_list=[]
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        for ability_upgrade in record['ability_upgrades']:
            train.loc[record['id'], 'upgrade_behavior {}'.format(
                ab.loc[ability_upgrade, 'behavior_first'])] += 1
        n_upgrades_list.append(len(record['ability_upgrades']))    
            
with open('academy2019_final_test.jsonlines') as fin:
    n_upgrades_list_t=[]
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        for ability_upgrade in record['ability_upgrades']:
            test.loc[record['id'], 'upgrade_behavior {}'.format(
                ab.loc[ability_upgrade, 'behavior_first'])] += 1
        n_upgrades_list_t.append(len(record['ability_upgrades'])) 

A Jupyter Widget

A Jupyter Widget

In [10]:
for dname in set(items.dname):
    train['final_item {}'.format(dname)] = 0
    test['final_item {}'.format(dname)] = 0
    
    
with open('academy2019_final_train.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        unfilled = 0
        record = json.loads(line)
        for item in record['final_items']:
            if item != 0:
                train.loc[record['id'], 'final_item {}'.format(items.loc[item, 'dname'])] += 1      
                

with open('academy2019_final_test.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        for item in record['final_items']:
            if item != 0:
                test.loc[record['id'], 'final_item {}'.format(items.loc[item, 'dname'])] += 1

A Jupyter Widget

A Jupyter Widget

In [12]:
for dname in set(items.dname):
    train['bought_item {}'.format(dname)] = 0
    test['bought_item {}'.format(dname)] = 0
    
    
with open('academy2019_final_train.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        for item in record['item_purchase_log']:
            item = item['item_id']
            if item != 0:
                train.loc[record['id'], 'bought_item {}'.format(items.loc[item, 'dname'])] += 1

                        
                        
with open('academy2019_final_test.jsonlines') as fin:
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        for item in record['item_purchase_log']:
            item = item['item_id']
            if item != 0:
                test.loc[record['id'], 'bought_item {}'.format(items.loc[item, 'dname'])] += 1

A Jupyter Widget

A Jupyter Widget

In [11]:
with open(academy2019_final_train.jsonlines') as fin:
    mean_for_team_list = []
    mean_gold_list=[]
    mean_diff_list=[]
    median_gold_list=[]
    diff_list=[]
    real_mean_list=[]
    part_of_all_list=[]
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        name = record['player_team']
        series = record['series']
        part_of_all = sum(series['player_gold'])/(sum(series['dire_gold'])+sum(series['radiant_gold']))
        mean_gold = sum(series['player_gold'])/len(series['player_gold'])
        median_gold = np.median(series['player_gold'])
        real_mean = sum(series['player_gold'])/sum(series[name+'_gold'])
        meaning = series['player_gold'][-1]/series[name+'_gold'][-1]
        diff = series['player_gold'][-1] - series[name+'_gold'][-1]
        mean_diff = (sum(series[name+'_gold'])/len(series[name+'_gold'])) - (sum(series['player_gold'])/len(series['player_gold']))
        
        mean_for_team_list.append(meaning)
        mean_gold_list.append(mean_gold)
        mean_diff_list.append(mean_diff)
        diff_list.append(diff)
        real_mean_list.append(real_mean)
        part_of_all_list.append(part_of_all)
        median_gold_list.append(median_gold)
        
        
with open('academy2019_final_test.jsonlines') as fin:
    mean_for_team_list_t = []
    mean_gold_list_t=[]
    mean_diff_list_t=[]
    diff_list_t=[]
    real_mean_list_t=[]
    part_of_all_list_t=[]
    median_gold_list_t=[]
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        name = record['player_team']
        series = record['series']
        part_of_all = sum(series['player_gold'])/(sum(series['dire_gold'])+sum(series['radiant_gold']))
        mean_gold = sum(series['player_gold'])/len(series['player_gold'])
        real_mean = sum(series['player_gold'])/sum(series[name+'_gold'])
        meaning = series['player_gold'][-1]/series[name+'_gold'][-1]
        diff = series['player_gold'][-1] - series[name+'_gold'][-1]
        median_gold = np.median(series['player_gold'])
        mean_diff = (sum(series[name+'_gold'])/len(series[name+'_gold'])) - (sum(series['player_gold'])/len(series['player_gold']))
        
        mean_for_team_list_t.append(meaning)
        mean_gold_list_t.append(mean_gold)
        mean_diff_list_t.append(mean_diff)
        diff_list_t.append(diff)
        real_mean_list_t.append(real_mean)
        part_of_all_list_t.append(part_of_all)
        median_gold_list_t.append(median_gold)

A Jupyter Widget

A Jupyter Widget

In [12]:
with open('academy2019_final_train.jsonlines') as fin:
    cost_list = []
    for line in tqdm.tqdm_notebook(fin):
        cost = 0
        record = json.loads(line)
        for final_item in record["final_items"]:
            if final_item != 0:
                cost += items.loc[final_item, 'cost']
        cost_list.append(cost)
        
        
with open('academy2019_final_test.jsonlines') as fin:
    cost_list_t = []
    for line in tqdm.tqdm_notebook(fin):
        cost = 0
        record = json.loads(line)
        for final_item in record["final_items"]:
            if final_item != 0:
                cost += items.loc[final_item, 'cost']
        cost_list_t.append(cost)

A Jupyter Widget

A Jupyter Widget

In [ ]:
with open('academy2019_final_train.jsonlines') as fin:
    apm_list = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        time = record['duration']/60
        apm = len(record['actions'])/time
        apm_list.append(cost)
        
        
with open('academy2019_final_test.jsonlines') as fin:
    apm_list_t = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        time = record['duration']/60
        apm = len(record['actions'])/time
        apm_list_t.append(cost)

In [ ]:
with open('academy2019_final_train.jsonlines') as fin:
    mean_dist_list = []
    max_dist_list = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        ac = record['actions']
        dist_l = []
        for i in range(len(ac['cursor_x'])-1)
            dist = np.sqrt((ac['cursor_x'][i+1] - ac['cursor_x'][i])**2 + (ac['cursor_y'][i+1] - ac['cursor_y'][i])**2)
            dist_l.append(dist)
        mean_dist_list.append(np.mean(dist_l))
        max_dist_list.append(np.max(dist_l))
        
with open('academy2019_final_test.jsonlines') as fin:
    mean_dist_list_t = []
    max_dist_list_t = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        ac = record['actions']
        dist_l = []
        for i in range(len(ac['cursor_x'])-1)
            dist = np.sqrt((ac['cursor_x'][i+1] - ac['cursor_x'][i])**2 + (ac['cursor_y'][i+1] - ac['cursor_y'][i])**2)
            dist_l.append(dist)
        mean_dist_list_t.append(np.mean(dist_l))
        max_dist_list_t.append(np.max(dist_l))

In [13]:
with open('academy2019_final_train.jsonlines') as fin:
    n_items_list = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        n_item = len(record["item_purchase_log"])
        n_items_list.append(n_item)
        
        
with open('academy2019_final_test.jsonlines') as fin:
    n_items_list_t = []
    for line in tqdm.tqdm_notebook(fin):
        record = json.loads(line)
        n_item = len(record["item_purchase_log"])
        n_items_list_t.append(n_item)

A Jupyter Widget

A Jupyter Widget

In [15]:
train["cost_final_items"] = cost_list
train['mean_for_team'] = mean_for_team_list
train['n_component'] = component
train['n_secret_shop'] = secret_shop
train['n_consumable'] = consumable
train['n_rare'] = rare
train['n_epic'] = epic
train['n_artifact'] = artifact
train['n_upgrades'] = n_upgrades_list
train['mean_gold'] = mean_gold_list 
train['mean_diff'] = mean_diff_list 
train['diff_gold'] = diff_list
train['real_mean'] = real_mean_list
train['part_of_all'] = part_of_all_list
train['median_gold'] = median_gold_list
train['n_items'] = n_items_list
train['apm'] = apm_list
train['mean_dist'] = mean_dist_list
train['max_dist'] = max_dist_list





test['n_upgrades'] = n_upgrades_list_t
test['n_empty_behavior'] = empty_t
test["cost_final_items"] = cost_list_t
test['mean_for_team'] = mean_for_team_list_t
test['n_component'] = component_t
test['n_secret_shop'] = secret_shop_t
test['n_consumable'] = consumable_t
test['n_rare'] = rare_t
test['n_epic'] = epic_t
test['n_artifact'] = artifact_t
test['mean_gold'] = mean_gold_list_t 
test['mean_diff'] = mean_diff_list_t 
test['diff_gold'] = diff_list_t
test['real_mean'] = real_mean_list_t
test['part_of_all'] = part_of_all_list_t
test['median_gold'] = median_gold_list_t
test['n_items'] = n_items_list_t
test['apm'] = apm_list_t
test['mean_dist'] = mean_dist_list_t
test['max_dist'] = max_dist_list_t

# Features from tables

In [16]:
train['in_winner_team'] = (train["player_team"]==train.winner_team).values.astype('int')
test['in_winner_team'] = (test["player_team"]==test.winner_team).values.astype('int')

In [17]:
attack_type = her.loc[train['hero_id'], 'attack_type']
primary = her.loc[train['hero_id'], 'primary_attr']
agi_gain = her.loc[train['hero_id'], 'agi_gain']
roles = her.loc[train['hero_id'], 'roles']
move_speed = her.loc[train['hero_id'], 'move_speed']


move_speed_t = her.loc[test['hero_id'], 'move_speed']
roles_t = her.loc[test['hero_id'], 'roles']
agi_gain_t = her.loc[test['hero_id'], 'agi_gain']
attack_type_t = her.loc[test['hero_id'], 'attack_type']
primary_t = her.loc[test['hero_id'], 'primary_attr']

In [20]:
train['attack_type'] = attack_type.values
train['primary_attr'] = primary.values
train['agi_gain'] = agi_gain.values
train['roles'] = roles.values
train['move_speed'] = move_speed.values




test['roles'] = roles_t.values
test['agi_gain'] = agi_gain_t.values
test['attack_type'] = attack_type_t.values
test['primary_attr'] = primary_t.values
test['move_speed'] = move_speed_t.values

In [21]:
hero_mean = train.loc[:, ['hero_id', "skilled"]].groupby("hero_id").agg("mean").reset_index() 
hero_mean.rename(columns={"skilled": "hero_mean"}, inplace=True) 
train= pd.merge(train, hero_mean, on=["hero_id"], how="left")
test= pd.merge(test, hero_mean, on=["hero_id"], how="left")

l_mean = train.loc[:, ['level', "skilled"]].groupby("level").agg("mean").reset_index() 
l_mean.rename(columns={"skilled": "lev_mean"}, inplace=True) 
train= pd.merge(train, l_mean, on=["level"], how="left")
test= pd.merge(test, l_mean, on=["level"], how="left")

pick_mean = train.loc[:, ['hero_pick_order', "skilled"]].groupby('hero_pick_order').agg("mean").reset_index() 
pick_mean.rename(columns={"skilled": "pick_mean"}, inplace=True) 
train= pd.merge(train, pick_mean, on=['hero_pick_order'], how="left")
test= pd.merge(test, pick_mean, on=['hero_pick_order'], how="left")

lst_mean = train.loc[:, ['leaver_status', "skilled"]].groupby("leaver_status").agg("mean").reset_index() 
lst_mean.rename(columns={"skilled": "lst_mean"}, inplace=True) 
train= pd.merge(train, lst_mean, on=["leaver_status"], how="left")
test= pd.merge(test, lst_mean, on=["leaver_status"], how="left")


win_mean = train.loc[:, ['in_winner_team', "skilled"]].groupby("in_winner_team").agg("mean").reset_index() 
win_mean.rename(columns={"skilled": "win_mean"}, inplace=True) 
test= pd.merge(test, win_mean, on=["in_winner_team"], how="left")
train = pd.merge(train, win_mean, on=["in_winner_team"], how="left")

att_mean = train.loc[:, ['attack_type', "skilled"]].groupby("attack_type").agg("mean").reset_index() 
att_mean.rename(columns={"skilled": "att_mean"}, inplace=True) 
train= pd.merge(train, att_mean, on=["attack_type"], how="left")
test= pd.merge(test, att_mean, on=["attack_type"], how="left")

roles_mean = train.loc[:, ['roles', "skilled"]].groupby("roles").agg("mean").reset_index() 
roles_mean.rename(columns={"skilled": "roles_mean"}, inplace=True) 
train= pd.merge(train, roles_mean, on=["roles"], how="left")
test= pd.merge(test, roles_mean, on=["roles"], how="left")

pr_mean = train.loc[:, ['primary_attr', "skilled"]].groupby('primary_attr').agg("mean").reset_index() 
pr_mean.rename(columns={"skilled": "pr_mean"}, inplace=True) 
train= pd.merge(train, pr_mean, on=["primary_attr"], how="left")
test= pd.merge(test, pr_mean, on=["primary_attr"], how="left")


agi_mean = train.loc[:, ['agi_gain', "skilled"]].groupby('agi_gain').agg("mean").reset_index() 
agi_mean.rename(columns={"skilled": "agi_mean"}, inplace=True) 
train= pd.merge(train, agi_mean, on=['agi_gain'], how="left")
test= pd.merge(test, agi_mean, on=['agi_gain'], how="left")

# Final preparations

In [22]:
cats = ['attack_type', 'primary_attr', 'roles']

for i in cats:
    train[i] = train[i].astype('category').cat.codes
    test[i] = test[i].astype('category').cat.codes

In [23]:
train = train.drop(['player_team', 'winner_team'], axis = 1)
test = test.drop(['player_team', 'winner_team'], axis = 1)

In [25]:
list(train.columns)

['duration',
 'pre_game_duration',
 'first_blood_time',
 'first_blood_claimed',
 'hero_id',
 'hero_pick_order',
 'leaver_status',
 'party_players',
 'kills',
 'deaths',
 'assists',
 'denies',
 'level',
 'net_worth',
 'gold',
 'gold_spent',
 'last_hits',
 'gold_per_min',
 'xp_per_min',
 'hero_damage',
 'tower_damage',
 'hero_healing',
 'scaled_hero_damage',
 'scaled_tower_damage',
 'scaled_hero_healing',
 'stuns',
 'team_fight_participation',
 'observer_wards_placed',
 'sentry_wards_placed',
 'creeps_stacked',
 'camps_stacked',
 'rune_pickups',
 'tower_kills',
 'roshan_kills',
 'nearby_creep_death_count',
 'radiant_tower_status',
 'dire_tower_status',
 'radiant_barracks_status',
 'dire_barracks_status',
 'fight_score',
 'farm_score',
 'support_score',
 'push_score',
 'avg_kills_x16',
 'avg_deaths_x16',
 'avg_assists_x16',
 'avg_gpm_x16',
 'avg_xpm_x16',
 'best_kills_x16',
 'best_assists_x16',
 'best_gpm_x16',
 'best_xpm_x16',
 'win_streak',
 'best_win_streak',
 'against_Chen',
 'with_Ch

# Список нужных колонок

In [25]:
 'bought_item nan',
 'bought_item Satanic',
 'bought_item Kaya',
 'bought_item Assault Cuirass Recipe',
 'bought_item Magic Wand',
 'bought_item Skull Basher',
 'bought_item Octarine Core',
 'bought_item Pocket Roshan',
 'bought_item Staff of Wizardry',
 'bought_item Spirit Vessel',
 'bought_item Morbid Mask',
 'bought_item Ring of Tarrasque',
 'bought_item Bottle',
 'bought_item Wraith Band',
 "bought_item Vladmir's Offering",
 'bought_item Headdress',
 'bought_item Guardian Greaves Recipe',
 'bought_item Mekansm Recipe',
 'bought_item Super Blink Dagger',
 'bought_item Dagon',
 "bought_item Linken's Sphere Recipe",
 'bought_item Slippers of Agility',
 'bought_item Armlet of Mordiggian Recipe',
 'bought_item Quarterstaff',
 'bought_item Sange and Yasha',
 'bought_item Iron Talon Recipe',
 'bought_item Phase Boots',
 'bought_item Necronomicon',
 'bought_item Iron Branch',
 'bought_item Arcane Boots',
 "bought_item Shiva's Guard",
 'bought_item Bracer',
 'bought_item Clarity',
 "bought_item Aghanim's Scepter",
 "bought_item Eul's Scepter Recipe",
 'bought_item Veil of Discord Recipe',
 'bought_item Ring of Protection',
 'bought_item Medallion of Courage',
 'bought_item Boots of Speed',
 'bought_item Animal Courier',
 'bought_item Urn of Shadows Recipe',
 'bought_item Reaver',
 'bought_item Tango',
 'bought_item Nullifier',
 'bought_item Yasha and Kaya',
 'bought_item Solar Crest',
 'bought_item Orchid Malevolence Recipe',
 'bought_item Gauntlets of Strength',
 'bought_item Null Talisman',
 'bought_item Force Staff Recipe',
 'bought_item Shadow Amulet',
 'bought_item Daedalus Recipe',
 'bought_item Necronomicon Recipe',
 'bought_item Mjollnir Recipe',
 'bought_item Ethereal Blade',
 'bought_item Abyssal Blade Recipe',
 "bought_item Heaven's Halberd",
 'bought_item Glimmer Cape',
 'bought_item Robe of the Magi',
 'bought_item Sacred Relic',
 'bought_item Sange',
 'bought_item Bloodthorn Recipe',
 'bought_item Bloodthorn',
 'bought_item Monkey King Bar',
 'bought_item Radiance',
 'bought_item Guardian Greaves',
 'bought_item River Vial: Chrome',
 'bought_item Blade Mail',
 'bought_item Pipe of Insight',
 'bought_item Holy Locket',
 'bought_item Yasha',
 'bought_item Crown',
 'bought_item River Vial: Slime',
 'bought_item Battle Fury Recipe',
 'bought_item Tombstone',
 'bought_item Eye of Skadi',
 'bought_item Helm of the Dominator',
 'bought_item Desolator',
 'bought_item Buckler Recipe',
 'bought_item Observer Ward',
 "bought_item Poor Man's Shield",
 'bought_item Circlet',
 'bought_item Orchid Malevolence',
 'bought_item Aegis of the Immortal',
 'bought_item Battle Fury',
 'bought_item Butterfly',
 'bought_item Talisman of Evasion',
 'bought_item Meteor Hammer',
 'bought_item Soul Ring Recipe',
 'bought_item Null Talisman Recipe',
 'bought_item Dagon Recipe',
 'bought_item Mask of Madness',
 'bought_item Drum of Endurance',
 'bought_item Ultimate Orb',
 'bought_item River Vial: Oil',
 'bought_item Pipe of Insight Recipe',
 'bought_item Boots of Travel Recipe',
 'bought_item Refresher Orb',
 'bought_item River Vial: Blood',
 'bought_item Smoke of Deceit',
 'bought_item Gem of True Sight',
 'bought_item Skull Basher Recipe',
 'bought_item Town Portal Scroll',
 'bought_item Force Staff',
 'bought_item Kaya and Sange',
 'bought_item Blades of Attack',
 "bought_item Eul's Scepter of Divinity",
 'bought_item Crystalys',
 'bought_item Sange Recipe',
 'bought_item Aeon Disk',
 'bought_item Daedalus',
 'bought_item Wind Lace',
 'bought_item Ring of Health',
 'bought_item Urn of Shadows',
 'bought_item Silver Edge',
 'bought_item Energy Booster',
 'bought_item Iron Talon',
 "bought_item Sage's Mask",
 "bought_item Linken's Sphere",
 'bought_item Tome of Knowledge',
 'bought_item Cloak',
 'bought_item Chainmail',
 'bought_item Void Stone',
 'bought_item Eaglesong',
 'bought_item Tango (Shared)',
 'bought_item Blink Dagger',
 'bought_item Infused Raindrops',
 'bought_item Observer and Sentry Wards',
 'bought_item River Vial: Potion',
 'bought_item Abyssal Blade',
 'bought_item Shadow Blade',
 'bought_item Helm of Iron Will',
 'bought_item Perseverance',
 'bought_item Heart of Tarrasque',
 'bought_item Crimson Guard Recipe',
 'bought_item Band of Elvenskin',
 'bought_item Divine Rapier',
 'bought_item Yasha Recipe',
 'bought_item Mantle of Intelligence',
 'bought_item Tranquil Boots',
 'bought_item Mjollnir',
 'bought_item Javelin',
 'bought_item Assault Cuirass',
 'bought_item Wraith Band Recipe',
 'bought_item Crystalys Recipe',
 'bought_item Magic Stick',
 'bought_item Lotus Orb',
 'bought_item Buckler',
 'bought_item Quelling Blade',
 "bought_item Shiva's Guard Recipe",
 'bought_item Hood of Defiance',
 'bought_item Hyperstone',
 'bought_item Manta Style Recipe',
 'bought_item Boots of Travel',
 'bought_item Radiance Recipe',
 'bought_item Oblivion Staff',
 'bought_item Scythe of Vyse',
 'bought_item Blight Stone',
 'bought_item Vitality Booster',
 'bought_item Belt of Strength',
 'bought_item River Vial: Electrified',
 'bought_item Kaya Recipe',
 'bought_item Hurricane Pike',
 'bought_item Point Booster',
 'bought_item Dragon Lance',
 'bought_item Ring of Basilius',
 'bought_item Enchanted Mango',
 'bought_item Armlet of Mordiggian',
 'bought_item Black King Bar',
 'bought_item Diffusal Blade',
 'bought_item Soul Booster',
 'bought_item Drum of Endurance Recipe',
 'bought_item Stout Shield',
 'bought_item Diffusal Blade Recipe',
 'bought_item Vanguard',
 'bought_item Moon Shard',
 'bought_item Maelstrom',
 'bought_item Ring of Aquila',
 'bought_item Refresher Shard',
 'bought_item Ghost Scepter',
 'bought_item Holy Locket Recipe',
 'bought_item Ogre Axe',
 'bought_item Heart of Tarrasque Recipe',
 'bought_item Hand of Midas Recipe',
 'bought_item Rod of Atos Recipe',
 'bought_item Aeon Disk Recipe',
 'bought_item Mekansm',
 'bought_item Echo Sabre',
 'bought_item Spirit Vessel Recipe',
 'bought_item Pocket Tower',
 'bought_item Bloodstone',
 'bought_item Cheese',
 'bought_item Demon Edge',
 'bought_item Aether Lens',
 'bought_item Broadsword',
 'bought_item Silver Edge Recipe',
 'bought_item Bracer Recipe',
 'bought_item Sentry Ward',
 'bought_item Hurricane Pike Recipe',
 'bought_item Healing Salve',
 'bought_item Power Treads',
 'bought_item Orb of Venom',
 'bought_item Dust of Appearance',
 'bought_item Manta Style',
 'bought_item Magic Wand Recipe',
 'bought_item River Vial: Dry',
 'bought_item Mystic Staff',
 'bought_item Faerie Fire',
 'bought_item Gloves of Haste',
 'bought_item Mithril Hammer',
 'bought_item Rod of Atos',
 'bought_item Blade of Alacrity',
 'bought_item Hand of Midas',
 'bought_item Refresher Orb Recipe',
 'bought_item Crimson Guard',
 'bought_item Aether Lens Recipe',
 'bought_item Black King Bar Recipe',
 'bought_item Soul Ring',
 'bought_item Veil of Discord',
 'bought_item Claymore',
 'bought_item Ring of Regen',
 'bought_item Platemail',
 'bought_item Headdress Recipe',
    
    
    
'duration',
 'pre_game_duration',
 'first_blood_time',
 'first_blood_claimed',
 'hero_id',
 'hero_pick_order',
 'leaver_status',
 'party_players',
 'kills',
 'deaths',
 'assists',
 'denies',
 'level',
 'net_worth',
 'gold',
 'gold_spent',
 'last_hits',
 'gold_per_min',
 'xp_per_min',
 'hero_damage',
 'tower_damage',
 'hero_healing',
 'scaled_hero_damage',
 'scaled_tower_damage',
 'scaled_hero_healing',
 'stuns',
 'team_fight_participation',
 'observer_wards_placed',
 'sentry_wards_placed',
 'creeps_stacked',
 'camps_stacked',
 'rune_pickups',
 'tower_kills',
 'roshan_kills',
 'nearby_creep_death_count',
 'radiant_tower_status',
 'dire_tower_status',
 'radiant_barracks_status',
 'dire_barracks_status',
 'fight_score',
 'farm_score',
 'support_score',
 'push_score',
 'avg_kills_x16',
 'avg_deaths_x16',
 'avg_assists_x16',
 'avg_gpm_x16',
 'avg_xpm_x16',
 'best_kills_x16',
 'best_assists_x16',
 'best_gpm_x16',
 'best_xpm_x16',
 'win_streak',
 'best_win_streak',
    
 'upgrade_behavior Hidden',
 'upgrade_behavior Unit Target',
 'upgrade_behavior No Target',
 'upgrade_behavior empty',
 'upgrade_behavior Point Target',
 'upgrade_behavior Passive',
 'upgrade_behavior AOE',
 'upgrade_behavior Channeled',
 'upgrade_behavior Instant Cast',
    
 'final_item nan',
 'final_item Scythe of Vyse',
 'final_item Monkey King Bar',
 'final_item Iron Talon Recipe',
 'final_item Nullifier',
 'final_item Refresher Shard',
 'final_item Skull Basher Recipe',
 'final_item Circlet',
 'final_item Aegis of the Immortal',
 'final_item Animal Courier',
 'final_item Dragon Lance',
 'final_item Radiance Recipe',
 'final_item Wind Lace',
 'final_item Drum of Endurance',
 'final_item Kaya',
 'final_item Gem of True Sight',
 'final_item Town Portal Scroll',
 'final_item Tranquil Boots',
 'final_item Morbid Mask',
 'final_item Platemail',
 'final_item Infused Raindrops',
 'final_item Battle Fury',
 'final_item Hand of Midas Recipe',
 'final_item Mask of Madness',
 "final_item Shiva's Guard",
 'final_item Observer and Sentry Wards',
 'final_item Radiance',
 'final_item Octarine Core',
 'final_item Claymore',
 'final_item Bloodthorn',
 'final_item Sange Recipe',
 'final_item Spirit Vessel Recipe',
 'final_item River Vial: Chrome',
 'final_item Hood of Defiance',
 'final_item Satanic',
 'final_item Heart of Tarrasque',
 'final_item Void Stone',
 'final_item Force Staff',
 'final_item Slippers of Agility',
 'final_item Ring of Tarrasque',
 "final_item Linken's Sphere",
 'final_item Enchanted Mango',
 'final_item Magic Wand Recipe',
 'final_item Echo Sabre',
 'final_item Vitality Booster',
 'final_item Magic Stick',
 'final_item Ring of Health',
 'final_item Iron Talon',
 'final_item Healing Salve',
 'final_item Urn of Shadows',
 'final_item Belt of Strength',
 'final_item Diffusal Blade',
 'final_item Armlet of Mordiggian Recipe',
 'final_item Hand of Midas',
 'final_item Abyssal Blade',
 'final_item Mithril Hammer',
 'final_item Gauntlets of Strength',
 'final_item Manta Style',
 'final_item Yasha and Kaya',
 'final_item Blade Mail',
 'final_item River Vial: Slime',
 'final_item Javelin',
 "final_item Poor Man's Shield",
 'final_item Power Treads',
 'final_item Energy Booster',
 'final_item Shadow Amulet',
 'final_item Mjollnir',
 'final_item Eye of Skadi',
 'final_item Tombstone',
 'final_item Ultimate Orb',
 'final_item Black King Bar Recipe',
 "final_item Eul's Scepter of Divinity",
 'final_item Assault Cuirass Recipe',
 'final_item Veil of Discord Recipe',
 'final_item Crimson Guard Recipe',
 'final_item Assault Cuirass',
 'final_item Silver Edge',
 'final_item River Vial: Electrified',
 'final_item Dagon Recipe',
 'final_item River Vial: Blood',
 'final_item Boots of Speed',
 'final_item Urn of Shadows Recipe',
 'final_item Refresher Orb Recipe',
 'final_item Force Staff Recipe',
 'final_item Medallion of Courage',
 'final_item Drum of Endurance Recipe',
 "final_item Shiva's Guard Recipe",
 'final_item Rod of Atos',
 'final_item Band of Elvenskin',
 'final_item Silver Edge Recipe',
 'final_item Buckler Recipe',
 'final_item Pocket Roshan',
 'final_item Crown',
 'final_item Demon Edge',
 'final_item Blink Dagger',
 'final_item Mjollnir Recipe',
 'final_item Ghost Scepter',
 'final_item Hyperstone',
 'final_item Super Blink Dagger',
 "final_item Heaven's Halberd",
 'final_item Sentry Ward',
 'final_item Cheese',
 'final_item Null Talisman Recipe',
 'final_item Hurricane Pike Recipe',
 'final_item Bottle',
 'final_item Aether Lens',
 'final_item Soul Booster',
 'final_item Necronomicon',
 'final_item Lotus Orb',
 'final_item Iron Branch',
 'final_item Kaya Recipe',
 'final_item Bracer',
 'final_item Blight Stone',
 'final_item Quarterstaff',
 'final_item Heart of Tarrasque Recipe',
 'final_item Quelling Blade',
 'final_item Faerie Fire',
 'final_item Solar Crest',
 'final_item Guardian Greaves',
 'final_item Diffusal Blade Recipe',
 'final_item Skull Basher',
 'final_item Glimmer Cape',
 'final_item Smoke of Deceit',
 'final_item Stout Shield',
 'final_item Orchid Malevolence Recipe',
 'final_item Headdress',
 'final_item Talisman of Evasion',
 'final_item Clarity',
 'final_item Pipe of Insight',
 'final_item Crimson Guard',
 'final_item Spirit Vessel',
 'final_item Orchid Malevolence',
 'final_item Moon Shard',
 'final_item Pipe of Insight Recipe',
 'final_item Broadsword',
 'final_item Observer Ward',
 "final_item Vladmir's Offering",
 'final_item Blade of Alacrity',
 'final_item Divine Rapier',
 'final_item Cloak',
 'final_item Tome of Knowledge',
 'final_item Butterfly',
 'final_item Mystic Staff',
 'final_item Bracer Recipe',
 'final_item Reaver',
 'final_item Desolator',
 'final_item Arcane Boots',
 'final_item Crystalys',
 'final_item Soul Ring Recipe',
 'final_item Ring of Aquila',
 'final_item Sange',
 'final_item Maelstrom',
 'final_item Holy Locket Recipe',
 'final_item Sacred Relic',
 'final_item Bloodstone',
 'final_item Ethereal Blade',
 'final_item River Vial: Dry',
 'final_item Vanguard',
 'final_item Manta Style Recipe',
 'final_item Hurricane Pike',
 'final_item Sange and Yasha',
 'final_item Ring of Basilius',
 'final_item Dagon',
 "final_item Aghanim's Scepter",
 'final_item Rod of Atos Recipe',
 'final_item Robe of the Magi',
 "final_item Linken's Sphere Recipe",
 'final_item Aether Lens Recipe',
 'final_item Shadow Blade',
 'final_item Necronomicon Recipe',
 'final_item Boots of Travel Recipe',
 'final_item Battle Fury Recipe',
 'final_item Guardian Greaves Recipe',
 'final_item Phase Boots',
 'final_item Tango',
 'final_item Ogre Axe',
 'final_item Mekansm',
 'final_item Yasha',
 'final_item River Vial: Potion',
 'final_item Soul Ring',
 'final_item Eaglesong',
 'final_item Daedalus Recipe',
 'final_item Yasha Recipe',
 'final_item Holy Locket',
 'final_item Wraith Band',
 'final_item Mekansm Recipe',
 "final_item Eul's Scepter Recipe",
 'final_item Helm of the Dominator',
 'final_item Aeon Disk Recipe',
 'final_item Blades of Attack',
 'final_item Staff of Wizardry',
 'final_item Armlet of Mordiggian',
 'final_item Meteor Hammer',
 'final_item Boots of Travel',
 'final_item Bloodthorn Recipe',
 'final_item Oblivion Staff',
 'final_item Kaya and Sange',
 'final_item Wraith Band Recipe',
 'final_item Point Booster',
 'final_item Daedalus',
 'final_item Veil of Discord',
 'final_item Aeon Disk',
 'final_item Headdress Recipe',
 'final_item Buckler',
 'final_item Helm of Iron Will',
 'final_item Black King Bar',
 'final_item Orb of Venom',
 'final_item Mantle of Intelligence',
 'final_item Crystalys Recipe',
 'final_item Refresher Orb',
 'final_item Null Talisman',
 'final_item Pocket Tower',
 "final_item Sage's Mask",
 'final_item Abyssal Blade Recipe',
 'final_item Gloves of Haste',
 'final_item Ring of Protection',
 'final_item Magic Wand',
 'final_item Perseverance',
 'final_item Chainmail',
 'final_item Tango (Shared)',
 'final_item Ring of Regen',
 'final_item Dust of Appearance',
 'final_item River Vial: Oil',
    
 'in_winner_team',
 'hero_mean',
 'lev_mean',
 'pick_mean',
 'lst_mean',
 'attack_type',
 'primary_attr',
 'att_mean',
 'pr_mean',
 'cost_final_items',
 "agi_mean",
 'roles_mean',
 'agi_gain',
 'roles',
 'move_speed',
    
 
 'n_component',
 'n_secret_shop',
 'n_consumable',
 'n_rare',
 'n_epic',
 'n_artifact',
    
 'against_Viper',
 'with_Viper',
 'against_Necrophos',
 'with_Necrophos',
 'against_Ursa',
 'with_Ursa',
 'against_Mars',
 'with_Mars',
 'against_Terrorblade',
 'with_Terrorblade',
 'against_Clockwerk',
 'with_Clockwerk',
 'against_Wraith King',
 'with_Wraith King',
 'against_Spirit Breaker',
 'with_Spirit Breaker',
 'against_Faceless Void',
 'with_Faceless Void',
 'against_Tinker',
 'with_Tinker',
 'against_Sven',
 'with_Sven',
 'against_Sniper',
 'with_Sniper',
 'against_Lone Druid',
 'with_Lone Druid',
 'against_Mirana',
 'with_Mirana',
 'against_Ember Spirit',
 'with_Ember Spirit',
 'against_Zeus',
 'with_Zeus',
 'against_Doom',
 'with_Doom',
 'against_Tusk',
 'with_Tusk',
 'against_Rubick',
 'with_Rubick',
 'against_Queen of Pain',
 'with_Queen of Pain',
 'against_Earth Spirit',
 'with_Earth Spirit',
 'against_Grimstroke',
 'with_Grimstroke',
 'against_Gyrocopter',
 'with_Gyrocopter',
 'against_Enigma',
 'with_Enigma',
 'against_Elder Titan',
 'with_Elder Titan',
 'against_Clinkz',
 'with_Clinkz',
 'against_Arc Warden',
 'with_Arc Warden',
 'against_Death Prophet',
 'with_Death Prophet',
 'against_Meepo',
 'with_Meepo',
 'against_Beastmaster',
 'with_Beastmaster',
 'against_Lion',
 'with_Lion',
 'against_Shadow Shaman',
 'with_Shadow Shaman',
 'against_Windranger',
 'with_Windranger',
 'against_Techies',
 'with_Techies',
 'against_Lich',
 'with_Lich',
 'against_Omniknight',
 'with_Omniknight',
 'against_Spectre',
 'with_Spectre',
 'against_Tiny',
 'with_Tiny',
 'against_Underlord',
 'with_Underlord',
 'against_Earthshaker',
 'with_Earthshaker',
 'against_Lina',
 'with_Lina',
 'against_Phantom Lancer',
 'with_Phantom Lancer',
 "against_Nature's Prophet",
 "with_Nature's Prophet",
 'against_Io',
 'with_Io',
 'against_Templar Assassin',
 'with_Templar Assassin',
 'against_Dark Willow',
 'with_Dark Willow',
 'against_Riki',
 'with_Riki',
 'against_Chaos Knight',
 'with_Chaos Knight',
 'against_Chen',
 'with_Chen',
 'against_Timbersaw',
 'with_Timbersaw',
 'against_Batrider',
 'with_Batrider',
 'against_Sand King',
 'with_Sand King',
 'against_Keeper of the Light',
 'with_Keeper of the Light',
 'against_Centaur Warrunner',
 'with_Centaur Warrunner',
 'against_Bounty Hunter',
 'with_Bounty Hunter',
 'against_Bristleback',
 'with_Bristleback',
 'against_Brewmaster',
 'with_Brewmaster',
 'against_Morphling',
 'with_Morphling',
 'against_Enchantress',
 'with_Enchantress',
 'against_Anti-Mage',
 'with_Anti-Mage',
 'against_Outworld Devourer',
 'with_Outworld Devourer',
 'against_Dragon Knight',
 'with_Dragon Knight',
 'against_Lifestealer',
 'with_Lifestealer',
 'against_Slardar',
 'with_Slardar',
 'against_Weaver',
 'with_Weaver',
 'against_Tidehunter',
 'with_Tidehunter',
 'against_Invoker',
 'with_Invoker',
 'against_Legion Commander',
 'with_Legion Commander',
 'against_Venomancer',
 'with_Venomancer',
 'against_Slark',
 'with_Slark',
 'against_Warlock',
 'with_Warlock',
 'against_Kunkka',
 'with_Kunkka',
 'against_Dark Seer',
 'with_Dark Seer',
 'against_Bane',
 'with_Bane',
 'against_Axe',
 'with_Axe',
 'against_Pudge',
 'with_Pudge',
 'against_Witch Doctor',
 'with_Witch Doctor',
 'against_Razor',
 'with_Razor',
 'against_Medusa',
 'with_Medusa',
 'against_Luna',
 'with_Luna',
 'against_Huskar',
 'with_Huskar',
 'against_Lycan',
 'with_Lycan',
 'against_Skywrath Mage',
 'with_Skywrath Mage',
 'against_Pangolier',
 'with_Pangolier',
 'against_Troll Warlord',
 'with_Troll Warlord',
 'against_Storm Spirit',
 'with_Storm Spirit',
 'against_Phoenix',
 'with_Phoenix',
 'against_Pugna',
 'with_Pugna',
 'against_Drow Ranger',
 'with_Drow Ranger',
 'against_Bloodseeker',
 'with_Bloodseeker',
 'against_Naga Siren',
 'with_Naga Siren',
 'against_Alchemist',
 'with_Alchemist',
 'against_Nyx Assassin',
 'with_Nyx Assassin',
 'against_Shadow Demon',
 'with_Shadow Demon',
 'against_Leshrac',
 'with_Leshrac',
 'against_Puck',
 'with_Puck',
 'against_Silencer',
 'with_Silencer',
 'against_Treant Protector',
 'with_Treant Protector',
 'against_Monkey King',
 'with_Monkey King',
 'against_Magnus',
 'with_Magnus',
 'against_Crystal Maiden',
 'with_Crystal Maiden',
 'against_Ogre Magi',
 'with_Ogre Magi',
 'against_Dazzle',
 'with_Dazzle',
 'against_Oracle',
 'with_Oracle',
 'against_Phantom Assassin',
 'with_Phantom Assassin',
 'against_Broodmother',
 'with_Broodmother',
 'against_Disruptor',
 'with_Disruptor',
 'against_Juggernaut',
 'with_Juggernaut',
 'against_Night Stalker',
 'with_Night Stalker',
 'against_Abaddon',
 'with_Abaddon',
 'against_Ancient Apparition',
 'with_Ancient Apparition',
 'against_Jakiro',
 'with_Jakiro',
 'against_Winter Wyvern',
 'with_Winter Wyvern',
 'against_Undying',
 'with_Undying',
 'against_Shadow Fiend',
 'with_Shadow Fiend',
 'against_Visage',
 'with_Visage',
 'against_Vengeful Spirit',
 'with_Vengeful Spirit',
    
 'n_upgrades',
               
 'mean_gold',
 'mean_for_team',
 'mean_diff',
 'diff_gold',
 'real_mean',
 'part_of_all',
 'median_gold',
               
    
'apm',    'mean_dist',
    'max_dist'

# Prediction

In [ ]:
X_train,X_test,y_train, y_test = train_test_split(train, y, random_state=6741, test_size=0.3)

X_train.shape, y_train.shape

In [27]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'verbose': 0,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['auc'],
    'num_leaves': 2 ** 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'min_child_weight': 20,
    'bagging_freq': 10,
     'seed': 7
                }

gbm = lgb.train(params, lgb_train,
            num_boost_round=10000,
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=20,
            early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.769521	valid_1's auc: 0.767753
[40]	training's auc: 0.799011	valid_1's auc: 0.797322
[60]	training's auc: 0.815586	valid_1's auc: 0.81203
[80]	training's auc: 0.827228	valid_1's auc: 0.822675
[100]	training's auc: 0.836201	valid_1's auc: 0.830763
[120]	training's auc: 0.84326	valid_1's auc: 0.837373
[140]	training's auc: 0.849884	valid_1's auc: 0.84321
[160]	training's auc: 0.855271	valid_1's auc: 0.847712
[180]	training's auc: 0.860368	valid_1's auc: 0.85238
[200]	training's auc: 0.86442	valid_1's auc: 0.855962
[220]	training's auc: 0.868462	valid_1's auc: 0.859384
[240]	training's auc: 0.872219	valid_1's auc: 0.862528
[260]	training's auc: 0.875367	valid_1's auc: 0.865129
[280]	training's auc: 0.878055	valid_1's auc: 0.867445
[300]	training's auc: 0.88052	valid_1's auc: 0.869666
[320]	training's auc: 0.883144	valid_1's auc: 0.87173
[340]	training's auc: 0.885041	valid_1's auc: 0.873022
[360]	trainin

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params1 = {
    'verbose': 0,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['auc'],
    'num_leaves': 2 ** 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'min_child_weight': 20,
    'bagging_freq': 10,
     'seed': 6764
                }

gbm1 = lgb.train(params1, lgb_train,
            num_boost_round=10000,
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=20,
            early_stopping_rounds=100)

# Blanding 

In [ ]:
y_pr = gbm.predict(test[useful_cols])
y_pr1 = gbm1.predict(test[useful_cols])

subm = 0.6*y_pr+0.4*y_pr1

submission_ = pd.DataFrame({'skilled_prob': subm}, index=index_test)

submission_.head(20)

In [ ]:
submission_.to_csv('submission1.csv')